In [1]:
import pandas as pd

# Load the Excel file
cols = ["보압시간", "사출속도1~4", "보압1~2"]
level_col_name = "변수 수준"
file_path = r"C:\Users\dcas\Documents\카카오톡 받은 파일\사출 실험계획표 27.xlsx"


epoch = 100
patience = 100
print_per_epoch = 1
input_params = cols
output_params = ["stress"]

excel_data = pd.read_excel(file_path)[[level_col_name] + cols]
# # find the index of the first NaN value
null_idx = excel_data.index[excel_data.iloc[:, 0].isna()].tolist()[0]

# Display the first few rows of the dataframe to understand its structure
level_table = excel_data.iloc[:null_idx, :].copy()
data_table = (
    excel_data[cols].iloc[null_idx + 2 :, :].copy().reset_index(drop=True)
)
levels = level_table[level_col_name].tolist()

# Display the first few rows of the dataframe to understand its structure
assert data_table.shape[1] == len(
    cols
), "The number of columns is not correct"
print(f"Levels: {levels}")
print(f"Data table shape: {data_table.shape}")
level_table.head()

Levels: [-1, 0, 1]
Data table shape: (27, 3)


,변수 수준,보압시간,사출속도1~4,보압1~2
0,-1,1.2,40,10
1,0,1.4,50,20
2,1,1.6,60,30


In [2]:
# Function to map actual values to level values (-1, 0, 1)
def map_to_level(value, column):
    # Find the corresponding level for the value in the specified column
    level = level_table[level_table[column] == value][level_col_name].values[
        0
    ]
    return level


# Initialize an empty DataFrame with the same shape as data_table
mapped_data = pd.DataFrame(columns=cols, index=range(len(data_table)))

# Map each column in data_table to its corresponding level
for col in cols:
    mapped_data[col] = data_table[col].apply(lambda x: map_to_level(x, col))

# Convert the DataFrame to integer type
mapped_data = mapped_data.astype(int)

# Set the index name to "Case"
mapped_data.index.name = "Case"

# Set the indices to start from 1
mapped_data.index = mapped_data.index.map(lambda x: x + 1)

# Display the first few rows of the mapped data
assert mapped_data.shape[1] == len(
    cols
), "The number of columns is not correct"
mapped_data.head()

,보압시간,사출속도1~4,보압1~2
Case,,,
1,-1,-1,-1
2,0,-1,-1
3,1,-1,-1
4,-1,-1,0
5,0,-1,0


In [5]:
from functools import reduce
import json
import multiprocessing
from pathlib import Path

from typing import Tuple
from uuid import uuid4
import numpy as np

import pandas as pd

from nn.ann import ANN
from nn.config import ANNModelConfig, LSTMModelConfig
from nn.dataloader import DataLoader
from nn.inference import inference
from nn.lstm import EmbeddingAttentionLSTMRegressor
from nn.schemas import (
    normalize_1d_sequence,
    _read_ss_curves,
    group_ss_curves,
)
from nn.train import Trainer
from nn.utils.logger import ApiLogger

logger = ApiLogger(__name__)


model_config = LSTMModelConfig(
    output_path=f".tmp/{uuid4().hex}",
    metrics=["mse", "mae"],
    kfold_splits=0,
    print_per_epoch=print_per_epoch,
    batch_size=1,
    epochs=epoch,
    patience=patience,
    loss_funcs=["mse"],
    loss_weights=[1.0],
    l1_reg=None,
    l2_reg=None,
    dropout_rate=0.0,
    normalize_layer=False,
    dim_out=1,
    seq_len=64,
    # ann_model_path="ANN_E10000[LR=0.001][N1=10][N2=10][N3=10].keras",
)
ss_curves = group_ss_curves(_read_ss_curves(raw_data_path=Path("data")))
ss_curves["Case"] = ss_curves.index.to_series().apply(
    lambda x: int(x.split("-")[1])
)
ss_curves = pd.merge(
    ss_curves.reset_index(drop=True),
    mapped_data,
    left_on="Case",
    right_index=True,
).drop(columns=["Case"])
encoder_inputs = ss_curves[cols].astype(float).to_numpy()
decoder_outputs = (
    ss_curves["stress"]
    .apply(
        lambda x: pd.Series(normalize_1d_sequence(x, model_config.seq_len))
    )
    .to_numpy()
)[:, :, np.newaxis]
decoder_inputs = np.zeros_like(decoder_outputs)
decoder_inputs[:, 1:, :] = decoder_outputs[:, :-1, :]  # Teacher forcing

assert encoder_inputs.shape[0] == decoder_outputs.shape[0], (
    f"Encoder input shape {encoder_inputs.shape} and decoder output shape {decoder_outputs.shape} "
    f"do not match"
)
encoder_inputs.shape, decoder_outputs.shape

((108, 3), (108, 64, 1))

In [6]:
train_inputs = encoder_inputs
train_outputs = decoder_outputs

data_loader = DataLoader(
    train_inputs=train_inputs,
    train_outputs=train_outputs,
    train_input_params=input_params,
    train_output_params=output_params,
)
trainer = Trainer(
    data_loader=data_loader,
    model_class=EmbeddingAttentionLSTMRegressor,
    model_name=EmbeddingAttentionLSTMRegressor.__name__,
    model_config=model_config,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=False,
)
train_inputs.shape, train_outputs.shape

((108, 3), (108, 64, 1))

In [7]:
import random


def test_data(
    train_inputs: np.ndarray, train_outputs: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    x_test, y_test = train_inputs, train_outputs
    assert isinstance(x_test, np.ndarray) and isinstance(
        y_test, np.ndarray
    ), f"{type(x_test)} & {type(y_test)}"
    assert (
        x_test.shape[0] == y_test.shape[0]
    ), f"{x_test.shape} != {y_test.shape}"
    random_idx = random.randint(0, x_test.shape[0] - 1)
    return (
        x_test[random_idx : random_idx + 1],
        y_test[random_idx : random_idx + 1],
    )


def test_inference(
    test_data: Tuple[np.ndarray, np.ndarray],
    model_path: str = r".tmp\172689c286e24684a2d2ba234ce454e6\LSTM_E556[SEQ_LEN=512].keras",
):
    x_test, y_test = test_data
    y_pred = inference(model_path, x_test)
    assert y_pred.shape == y_test.shape, f"{y_pred.shape} != {y_test.shape}"
    seq_len = y_pred.shape[1]
    n = 5

    def extract_points(y: np.ndarray):
        gap = seq_len // (n - 1)
        last_idx = seq_len - 1
        return tuple(y[0, min(i * gap, last_idx), 0] for i in range(n))

    y_pred_points = extract_points(y_pred)[1:]
    y_test_points = extract_points(y_test)[1:]
    print(f"prediction: {y_pred_points}, true: {y_test_points}")
    for yp, yt in zip(y_pred_points, y_test_points):
        assert abs(yp - yt) <= yt * 0.5, f"{yp} != {yt}"


all_hyper_params = {"seq_len": (model_config.seq_len,)}
num_hyper_params = reduce(
    lambda x, y: x * len(y), all_hyper_params.values(), 1
)
for fstem, phist in trainer.hyper_train(all_hyper_params):
    num_hyper_params -= 1
    json.dumps(phist["train_output"], indent=4)
    test_inference(
        model_path=fstem + ".keras",
        test_data=test_data(train_inputs, train_outputs),
    )
assert num_hyper_params == 0, f"{num_hyper_params} != 0"

[2023-11-23 21:26:50,230] nn.train:CRITICAL - model: EmbeddingAttentionLSTMRegressor with 1 cases
[2023-11-23 21:26:50,231] nn.train:CRITICAL - training without multiprocessing...
[2023-11-23 21:26:50,324] nn.train:INFO - Start training: LSTMModelConfig(seed=777, print_per_epoch=1, output_path='.tmp/5c4dd40824674f728da746b0cee43fb7', metrics=['mse', 'mae'], epochs=100, batch_size=1, kfold_splits=0, patience=100, dim_in=10, dim_out=1, lr=0.001, loss_funcs=['mse'], loss_weights=[1.0], activation='relu', l1_reg=None, l2_reg=None, dropout_rate=0.0, normalize_layer=False, freeze_layers=[], seq_len=64, ann_model_path=None, state_transform_activation='tanh')
[2023-11-23 21:26:54,931] nn.callbacks:DEBUG - [Epoch   1  ]	rmse: 0.34850	loss: 0.12145	mse: 0.12145	mae: 0.22653	val_loss: 0.00743	val_mse: 0.00743	val_mae: 0.05719
[2023-11-23 21:26:57,264] nn.callbacks:DEBUG - [Epoch   2  ]	rmse: 0.08177	loss: 0.00669	mse: 0.00669	mae: 0.04666	val_loss: 0.00621	val_mse: 0.00621	val_mae: 0.04440
[2023-